In [1]:
import face_recognition
import re
import os
import numpy as np
from tkinter import *
from PIL import Image, ImageTk

필요한 라이브러리들을 import합니다.

In [2]:
def get_cropped_face(image_file):
    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)
    a, b, c, d = face_locations[0]
    cropped_face = image[a:c,d:b,:]
    
    return cropped_face

원활한 얼굴 인식을 위해 얼굴 부위만 잘라주는 함수를 정의합니다.

In [3]:
def get_face_embedding(face):
    return face_recognition.face_encodings(face)[0]

In [4]:
def get_face_embedding_dict(dir_path):
    file_list = os.listdir(dir_path)
    embedding_dict = {}
    
    for file in file_list:
        image_file = os.path.join(dir_path, file)
        face = get_cropped_face(image_file)
        embedding_dict[re.sub(r"\..+", "", file)] = get_face_embedding(face)
        
    return embedding_dict

쉽게 비교하기 위해 얼굴 벡터 정보를 딕셔너리에 넣어줍니다.

In [5]:
def get_distance(name1, name2, embedding_dict):
    return np.linalg.norm(embedding_dict[name1]-embedding_dict[name2], ord=2)

두 얼굴 벡터 사이의 차이를 구하는 함수를 정의합니다.

In [6]:
def get_nearest_face(name, embedding_dict, top=5):
    distance_dict = {key: get_distance(name, key, embedding_dict) for key in embedding_dict.keys()}
    i = 1
    for key, val in sorted(distance_dict.items(), key=lambda item: item[1]):
        if not re.match(key[:-1], name):
            print(f"순위 {i}: 이름 ({key}), 거리({val})")
            if i is top:
                break
            i += 1

가장 닮은 얼굴 순위를 정해주는 함수를 정의합니다.

In [7]:
dir_path = os.getenv('HOME')+'/AIFFEL/2021-10-07/images'
embedding_dict = get_face_embedding_dict(dir_path)

In [8]:
get_nearest_face("jihyun2",embedding_dict)

순위 1: 이름 (mina), 거리(0.42425971050483674)
순위 2: 이름 (boyoung), 거리(0.44283604887891936)
순위 3: 이름 (yuna), 거리(0.4542379258195572)
순위 4: 이름 (goeun), 거리(0.4769009722058174)
순위 5: 이름 (sodam), 거리(0.4780564574584746)


In [9]:
get_nearest_face("jihyun1",embedding_dict)

순위 1: 이름 (sodam), 거리(0.4223197401376081)
순위 2: 이름 (mina), 거리(0.4305613356589147)
순위 3: 이름 (goeun), 거리(0.44320246105220495)
순위 4: 이름 (yuna), 거리(0.45273202918991917)
순위 5: 이름 (yeri), 거리(0.48262088204750647)


제 자신을 제외하고 가장 닮은 얼굴 순위를 뽑아보았습니다. 걸스데이 민아, 배우 김고은, 배우 박소담, 피겨스케이트 선수 김연아 이렇게 공통적으로 닮았다고 나오는군요!

# 재미있는 시각화

누가 얼만큼 닮았는지를 표처럼 나타내 보았습니다.

In [10]:
window = Tk()
window.title("누가 누가 닮았나")
window.config(padx=20, pady=20)
thumbnail_dir = os.getenv('HOME')+'/AIFFEL/2021-10-07/thumbnails'

우선 `tkinter` 윈도우를 만들고 얼굴 사진들의 디렉토리를 지정합니다.

In [11]:
# for file in os.listdir(dir_path):
#     file_path = os.path.join(dir_path, file)
#     face = get_cropped_face(file_path)
#     pillow_image = Image.fromarray(face)
#     pillow_image = pillow_image.resize((50,50))
#     pillow_image.save(os.path.join(thumbnail_dir, file))

그리고 얼굴 이미지들을 불러와서 리사이즈 해서 저장합니다. 이 코드는 한 번만 실행하면 다 저장되므로 주석 처리 해두었습니다.

In [12]:
faces_list = os.listdir(thumbnail_dir)
faces_list.insert(0, None)
content = []
files = []
for k in range(len(faces_list)):
    content.insert(k, [])
for i, file1 in enumerate(faces_list):
    for j, file2 in enumerate(faces_list):
        if j == 0 and i == 0:
            content[i].insert(j, Label(text="숫자가\n작을수록\n닮았습니다."))
        elif j == 0:
            content[i].insert(j, Canvas(height=50, width=50))
            file1_image = ImageTk.PhotoImage(file=os.path.join(thumbnail_dir, file1))
            files.append(file1_image)
            content[i][j].create_image(25, 25, image=file1_image)
        elif i == 0:
            content[i].insert(j, Canvas(height=50, width=50))
            file2_image = ImageTk.PhotoImage(file=os.path.join(thumbnail_dir, file2))
            files.append(file2_image)
            content[i][j].create_image(25, 25, image=file2_image)
        else:
            content[i].insert(j, Label(text=str(round(get_distance(re.sub(r"\..+", "", file1), re.sub(r"\..+", "", file2), embedding_dict), 2))))
        content[i][j].grid(row=i, column=j)

그리고 반복문을 통해 얼굴마다 사진을 출력하고 각기 다른 얼굴들의 차이를 출력합니다.

In [13]:
window.mainloop()

`tkinter` 윈도우를 켜놓기 위한 코드입니다.

## 시각화 결과
![시각화 결과](screenshot.png)

시각화 결과는 다음과 같이 나옵니다.

# 회고

**어려웠던 점**: 재미있는 시각화 아이디어를 내는 것이 어려웠고 tkinter를 쓴지 오래되어 다시 익히는 것이 어려웠습니다.

**알아낸 것**: face_recognition 라이브러리에 대해서 더 배우게 되었습니다.

**아직 모호한 점**: L2, L1 distance의 개념을 아직 정확히 이해하고 있지 않은 것 같습니다.

**평가 지표를 맞추기 위해 시도한 것들**: 재미있는 시각화를 하기 위해 tkinter를 사용하였습니다.

**다짐**: Norm distance에 대해 더 알아봐야겠습니다.